# MarketPlace SDK

In order to easily and effectively communicate with a registered application in MarketPlace via Python, the Python Software Development Kit (SDK) can be used. Follow the instructions on the  [application registration](../apps/registration.md) page to register an application with MarketPlace. Once the application is registered, you will find the `client_id` in the registration output.

The SDK is based on the [standard api](../apps/mp-api.md). For further information about installation and usage, visit the [Python SDK repository](https://github.com/materials-marketplace/python-sdk).

## Installation

We recommend using [virtual environment](https://docs.python.org/3/tutorial/venv.html) or an environment manager like [conda](https://docs.conda.io/en/latest/).
Once setup, you can simply install the SDK with the following command (`%` is needed for the notebook):

In [ ]:
%pip install marketplace-sdk

## How to call capabilities using the SDK?

The SDK consists of all the available capabilities implemented as python functions.
It enables users to create a python instance of a registered application and simply call the supported capabilities associated with it via class methods with appropriate inputs.
Users can then communicate with applications without having to worry about headers, requests and responses.
The following example is from a transformation application, i.e. it only supports transformation capabilities.

Remember that an application must be purchased in the platform in order to be used.

The SDK requires two parameters to connect to the MarketPlace. These are the path to the MarketPlace instance, and an access token belonging to a user. Your token can be accessed in the _Advanced_ section of your MarketPlace profile.

We recommend that you set these values as environment variables (`MP_HOST` and `MP_ACCESS_TOKEN` respectively). Note that the MarketPlace instance corresponds to the main deployment (https://www.materials-marketplace.eu/), so it is not necessary to define it.

In [ ]:
%export MP_ACCESS_TOKEN="<your access token>"

In [5]:
from marketplace.app import get_app

mp = get_app(app_id="<registered application's client_id>")

print(mp.heartbeat())

SimPARTIX-App : application running.


Example 1: starting a new transformation by calling the `newTransformation` capability with a configuration.
The POST request is created by the SDK automatically and sent with the payload

In [8]:
from marketplace_standard_app_api.models.transformation import NewTransformationModel, TransformationId

transformation_id: TransformationId = mp.new_transformation(parameters={"some_parameter": "some_value"}).id
transformation_id

'ec0f220f-c9e9-4d70-ad52-663d6441668a'

You can now start the transformation with this `transformation_id`.

In [9]:
mp.start_transformation(transformation_id=transformation_id)

"Simulation 'ec0f220f-c9e9-4d70-ad52-663d6441668a' started successfully!"

Remember that you can only access capabilities that are supported by the application.
The application will support specific capabilities specified in the openAPI file used to register the application.
Trying to access any unsupported capabilities throws a `NotImplemented` error.

In [10]:
mp.list_collections()

NotImplementedError: The app does not support this capability.

## How to modify capabilities with custom implementations?

Sometimes you will need to validate or preprocess user inputs before sending the request to the MarketPlace.
You might also require some post-processing of the responses.
This all can be done by creating a class that inherits from `MarketPlaceApp` and overrides the necessary methods.

In [11]:
from marketplace.app.v0 import MarketPlaceApp
class MyMarketPlaceApp(MarketPlaceApp):
    def heartbeat(self) -> str:
        return f"My app says hi: {super().heartbeat()}"

You can now use this class to carry out these custom functionalities just defined.

In [13]:

my_mp_app = MyMarketPlaceApp(app_id="<registered application's client_id>")

transformation_id = my_mp_app.new_transformation(NewTransformationModel(parameters=config)).id
# Since this capability was not overridden, this will return the standard response

my_mp_app.heartbeat()

'My app says hi: SimPARTIX-App : application running.'